## NLP Setup

In [1]:
import pickle
import sys
import spacy
from spacy import displacy
import benepar

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

## Set up identifiers

In [4]:
from app.src.sentence_summarizer import SentenceSummarizer
summarizer = SentenceSummarizer(nlp)

In [5]:
from app.src.primitive_identifiers.dicts.event_dict_set_builder import EventDictSetBuilder

all_event_dicts = EventDictSetBuilder.build_all()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
from app.src.primitive_extractor import PrimitiveExtractor
from app.src.primitive_identifiers.obligation_event_identifier import ObligationEventIdentifier
from app.src.primitive_identifiers.power_event_identifier import PowerEventIdentifier
from app.src.primitive_identifiers.contract_event_identifier import ContractEventIdentifier
from app.src.primitive_identifiers.obligation_state_identifier import ObligationStateIdentifier
from app.src.primitive_identifiers.power_state_identifier import PowerStateIdentifier
from app.src.primitive_identifiers.contract_state_identifier import ContractStateIdentifier
from app.src.primitive_identifiers.event_scorer import EventScorer
from app.src.primitive_identifiers.primitive_identifier import IIdentifyPrimitives
from app.src.primitive_identifiers.state_name_scorer import StateNameScorer
from app.src.primitive_identifiers.variable_scorer import VariableScorer
from app.src.primitive_identifiers.similarity_calculator import SentenceSimilarityCalculator
from app.src.primitive_identifiers.sentence_preprocessor import SentencePreprocessor
from app.templates.meat_sale.templates import meat_sale_templates

preprocessor = SentencePreprocessor(nlp)
similarity_calculator = SentenceSimilarityCalculator(preprocessor)

# ObligationEvent
ob_event_scorer = EventScorer(all_event_dicts['obligation'])
obligation_sent_dict = meat_sale_templates['obligations']
ob_var_scorer = VariableScorer(nlp, obligation_sent_dict, similarity_calculator)
oe_identifier = ObligationEventIdentifier(ob_event_scorer, ob_var_scorer)

# PowerEvent
power_event_scorer = EventScorer(all_event_dicts['power'])
power_sent_dict = meat_sale_templates['powers']
power_var_scorer = VariableScorer(nlp, power_sent_dict, similarity_calculator)
pe_identifier = PowerEventIdentifier(power_event_scorer, power_var_scorer)

# ContractEvent
c_event_scorer = EventScorer(all_event_dicts['contract'])
ce_identifier = ContractEventIdentifier(c_event_scorer)

ident_list: list[IIdentifyPrimitives] = [
    oe_identifier,
    pe_identifier,
    ce_identifier,
]

primitive_extractor = PrimitiveExtractor(ident_list)


In [8]:

test_sentence = 'seller has resumed delivery of goods'

summarizer.summarize(test_sentence)

doc = nlp(test_sentence)

results = primitive_extractor.extract(doc)

for x in ident_list:
    next_res = x.identify(doc)
    print(type(next_res.primitive))
    print(next_res.score)


c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


<class 'app.classes.spec.sym_event.ObligationEvent'>
0.797
<class 'app.classes.spec.sym_event.PowerEvent'>
0.765
<class 'app.classes.spec.sym_event.ContractEvent'>
0.7
